## Oil mining company

We have data on oil wells in three regions. The goal of the project is to find the best place for a new well. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#### Region 0

In [2]:
df_0=pd.read_csv('/datasets/geo_data_0.csv')
df_0.info()
df_0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
df_0[df_0.duplicated(keep=False)]

,id,f0,f1,f2,product


In [4]:
df_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


#### Region 1

In [5]:
df_1=pd.read_csv('/datasets/geo_data_1.csv')
df_1.info()
df_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
df_1[df_1.duplicated(keep=False)]

,id,f0,f1,f2,product


In [7]:
df_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


#### Region 2

In [8]:
df_2=pd.read_csv('/datasets/geo_data_2.csv')
df_2.info()
df_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [9]:
df_2[df_2.duplicated(keep=False)]

,id,f0,f1,f2,product


In [10]:
df_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


We have three datasets with 100,000 entries each. There no missing values, duplicated rows. Data type of each column is ok. Mean and median values are close to each other.

Our target variable ("product" - volume of reserves in the oil well ) is numerical, so we have regression task. 

There is no relationship between "id" and "product". So we can drop this column in all three datasets.

In [11]:
df_0=df_0.drop(['id'], axis=1)
df_1=df_1.drop(['id'], axis=1)
df_2=df_2.drop(['id'], axis=1)

Let's train linear regression for each region.  

* Region 0

In [12]:
target = df_0['product']
features = df_0.drop('product', axis=1)

# split the data into a training set and validation set at a ratio of 75:25
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    
# scale the numeric data using StandardScaler() from sklearn.preprocessing module.
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
    
# train Linear Regression
regressor = LinearRegression()  
regressor.fit(features_train, target_train)
predicted_valid = regressor.predict(features_valid)

rmse=mean_squared_error(target_valid, predicted_valid)**0.5
print('RMSE region 0:', rmse)

RMSE region 0: 37.5794217150813


Let's save the predictions and correct answers for the validation set and find the average volume of predicted reserves.

In [13]:
target_valid_0=target_valid

predicted_valid_0=pd.Series(predicted_valid,index=target_valid.index)
avg_vol_predicted_0 = predicted_valid_0.mean()
avg_vol_predicted_0 

92.59256778438038

* Region 1

In [14]:
target = df_1['product']
features = df_1.drop('product', axis=1)

# split the data into a training set and validation set at a ratio of 75:25
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    
# scale the numeric data using StandardScaler() from sklearn.preprocessing module.
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
    
# train Linear Regression
regressor = LinearRegression()  
regressor.fit(features_train, target_train)
predicted_valid = regressor.predict(features_valid)

rmse=mean_squared_error(target_valid, predicted_valid)**0.5
print('RMSE region 1:', rmse)

RMSE region 1: 0.8930992867756158


In [15]:
target_valid_1=target_valid

predicted_valid_1=pd.Series(predicted_valid,index=target_valid.index)
avg_vol_predicted_1 = predicted_valid_1.mean()
avg_vol_predicted_1 

68.728546895446

* Region 2

In [16]:
target = df_2['product']
features = df_2.drop('product', axis=1)

# split the data into a training set and validation set at a ratio of 75:25
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    
# scale the numeric data using StandardScaler() from sklearn.preprocessing module.
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
    
# train Linear Regression
regressor = LinearRegression()  
regressor.fit(features_train, target_train)
predicted_valid = regressor.predict(features_valid)

rmse=mean_squared_error(target_valid, predicted_valid)**0.5
print('RMSE region 2:', rmse)

RMSE region 2: 40.02970873393434


In [17]:
target_valid_2=target_valid

predicted_valid_2=pd.Series(predicted_valid,index=target_valid.index)
avg_vol_predicted_2 = predicted_valid_2.mean()
avg_vol_predicted_2 

94.96504596800489

In [18]:
summary = pd.DataFrame({'Region 0' : [37.58, 92.59],'Region 1' : [0.89, 68.73],'Region 2' : [40.02, 94.96]}, index=['RMSE','Mean volume predicted'])
summary

,Region 0,Region 1,Region 2
RMSE,37.58,0.89,40.02
Mean volume predicted,92.59,68.73,94.96


RMSE is a good measure of how accurately the model predicts the response. Lower values of RMSE indicate better fit.

The best value of RMSE is in region 1. If we are looking at average volume of predicted reserves, we can see that the region 2 is a leader. 

Now let's calculate the volume of reserves sufficient for developing a new well without losses. 

* The budget for development of 200 oil wells is 100 USD million.
* The revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels).

volume*revenue from one unit of product - budget for development of 200 oil wells/200 > 0 

In [19]:
rev = 4500 
budget_200_wells = 100000000

In [20]:
volume_without_losses=(budget_200_wells/200)/rev
volume_without_losses

111.11111111111111

In oder to develop a new well without losses It must have more than 112,000 barrels reserves.

From describe() function we can find the average volume of reserves in each region.

Region 0: average volume of reserves = 92.500 (volume of reserves is in thousand barrels)

Region 1: average volume of reserves = 68.825

Region 2: average volume of reserves = 95.000    

Region 2 has the highest average volume of reserves, but still it is not enough to cover losses.

We would like to pick 200 wells with the highest values of predictions for each region. 

Let's write a function that calculates profit from a set of selected oil wells and model predictions. The function will pick the wells with the highest values of predictions, then based on true volume of reserves of those wells, calculate the profit. 

In [21]:
def profit(target,predicted,count):
    predicted_sorted = predicted.sort_values(ascending=False)
    selected = target[predicted_sorted.index][:count]
    return 4500*selected.sum() - (100000000/200)*count 

Now we'll use the bootstrapping technique with 1000 samples to find the distribution of profit. We will specify subsample size n=500. Then we'll find average profit, 95% confidence interval and risk of losses for each region. Loss is a negative profit. 

* Region 0

In [22]:
state = np.random.RandomState(12345)

# From 500 wells picking the best 200 for the profit calculation.
values = []
for i in range(1000):
    target_subsample = target_valid_0.sample(n=500, replace=True, random_state=state)
    predicted_subsample = predicted_valid_0[target_subsample.index]
    
    values.append(profit(target_subsample,predicted_subsample,200))
    
# Average profit    
values = pd.Series(values)
mean = values.mean()

# 95% confidence interval
lower = values.quantile(.025)
upper = values.quantile(.975)

# Percentage of loss 
loss= len(list(filter(lambda x: x<0, values)))/1000*100

print("Average profit:", mean)
print("2.5% quantile:", lower)
print("97.5% quantile:", upper)
print("Percentage of loss:", loss)

Average profit: 4259385.269105923
2.5% quantile: -1020900.9483793724
97.5% quantile: 9479763.533583675
Percentage of loss: 6.0


* Region 1

In [23]:
values = []
for i in range(1000):
    target_subsample = target_valid_1.sample(n=500, replace=True, random_state=state)
    predicted_subsample = predicted_valid_1[target_subsample.index]
    
    values.append(profit(target_subsample, predicted_subsample,200))
    
values = pd.Series(values)
mean = values.mean()

lower = values.quantile(.025)
upper = values.quantile(.975)

loss= len(list(filter(lambda x: x<0, values)))/1000*100

print("Average profit:", mean)
print("2.5% quantile:", lower)
print("97.5% quantile:", upper)
print("Percentage of loss:", loss)

Average profit: 5182594.93697325
2.5% quantile: 1281232.3143308603
97.5% quantile: 9536129.820669085
Percentage of loss: 0.3


* Region 2

In [24]:
values = []
for i in range(1000):
    target_subsample = target_valid_2.sample(n=500, replace=True, random_state=state)
    predicted_subsample = predicted_valid_2[target_subsample.index]
    
    values.append(profit(target_subsample, predicted_subsample,200))
    
values = pd.Series(values)
mean = values.mean()

lower = values.quantile(.025)
upper = values.quantile(.975)

loss= len(list(filter(lambda x: x<0, values)))/1000*100

print("Average profit:", mean)
print("2.5% quantile:", lower)
print("97.5% quantile:", upper)
print("Percentage of loss:", loss)

Average profit: 4201940.053440501
2.5% quantile: -1158526.091600102
97.5% quantile: 9896299.398445744
Percentage of loss: 6.2


Let's summarize our results.

In [25]:
summary = pd.DataFrame({'Region 0' : [4259385, -1020901, 9479763, 6.0],'Region 1' : [5182595, 1281232, 9536130, 0.3],'Region 2' : [4201940, -1158526, 9896299, 6.2]}, index=['Average profit','2.5% quantile','97.5% quantile','Percentage of loss'])
summary

,Region 0,Region 1,Region 2
Average profit,4259385.0,5182595.0,4201940.0
2.5% quantile,-1020901.0,1281232.0,-1158526.0
97.5% quantile,9479763.0,9536130.0,9896299.0
Percentage of loss,6.0,0.3,6.2


We see that region 1 has the smalest percentage of losses (only 0.3%), as well as the highest average profit among all other regions. We are 95% confident that profit from 200 wells in Region 1 is between 1,281,232 and 9,536,130 USD.    

#### Thus, we can suggest to choose a region 1 as the best region for the development of oil wells.